# Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (in sklearn, obviously). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the next exercise, you will implement your own LASSO solver, using coordinate descent. 

## The usual

In [0]:
import sklearn
import pandas
import numpy as np

## Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located. *I am so surprised.*

In [0]:
full_data = pandas.read_csv("kc_house_data.csv", index_col=0)

## Create new features

As in Lab 2 (*lab-2.ipynb*), we consider features that are some transformations of inputs.

In [0]:
from math import log, sqrt
full_data['sqft_living_sqrt'] = full_data['sqft_living'].map(sqrt)
full_data['sqft_lot_sqrt'] = full_data['sqft_lot'].map(sqrt)
full_data['bedrooms_square'] = full_data['bedrooms'] ** 2

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
full_data['floors'] = full_data['floors'].astype(float) 
full_data['floors_square'] = full_data['floors'] ** 2

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [0]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`alpha=l1_penalty`) to the sklearn model `Lasso`. (Other tools may have separate implementations of LASSO). Much like L2/Ridge Regression, the features should be scaled to ensure equal attention inbetween.

In [0]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
l1_penalty=5e4
full_features = scaler.fit_transform(full_data[all_features].values)
full_labels = full_data['price'].values
model = Lasso(alpha=l1_penalty).fit(full_features, full_labels)

Find what features had non-zero weight.

In [0]:
# Do you know that even numpy has built-in boolean selector?

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test (9/1)
* Further split our training data into two sets: train, validation (5/5)

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [0]:
from sklearn.model_selection import train_test_split
# Cookie for those who instantly remember what cell to copy

Next, we write a loop that does the following:
* For `l1_penalty` in 21 steps range between [1, 10^9] (use `np.logspace(0, 9, num=21)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `alpha=l1_penalty` in the parameter.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

In [0]:
# Handholding level: Nominally.


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [0]:
# Eyeball power, or computation power?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [0]:
# Interesting, isn't it

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 5 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [0]:
max_nonzeros = 5

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [0]:
l1_penalty_values = np.logspace(3, 5, num=21)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(3, 5, num=21)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `alpha=l1_penalty` in the parameter.
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coef_` gives you the coefficients/parameters you learned (barring intercept) in the form of numpy array. You can then use array\[condition\] for the list of values passing the condition. Or just use the builtin `np.count_nonzero()`

In [0]:
# Or you can do it with pure python. Its impact on the performance is negligible.

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The smallest `l1_penalty` that has non-zeros equal `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The biggest `l1_penalty` that has non-zeros equal `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [0]:
l1_penalty_min = #
l1_penalty_max = #

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [0]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `alpha=l1_penalty`.
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [0]:
# See the quiz below as well

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?